In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from matminer.featurizers.structure import XRDPowderPattern
import plotly.express as px

In [1]:

def tsne_algo(X, perplexity, labels_true):
    
    from sklearn.manifold import TSNE
    n_components = 3
    Xtsne = TSNE(n_components, perplexity=perplexity).fit_transform(X)
    dftsne = pd.DataFrame(Xtsne)
    dftsne['labels'] = labels_true
    return dftsne


In [58]:
def kmeans_algo(df):
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=4)
    labels = kmeans.fit_predict(df)
    df['cluster'] = labels
    df['labels'] = labels_true
    return df

In [3]:
def sort_clusterlabels(dftsne_kmeans, label_count):
    dftsne_times = dftsne_kmeans.groupby(["labels", "cluster"]).size().reset_index(name="Time")
    
    m_df = dftsne_times[dftsne_times['labels'].str.match('m')]
    po_df = dftsne_times[dftsne_times['labels'].str.match('p-o')]
    o_df = dftsne_times[dftsne_times['labels'].str.match('o')]
    t_df = dftsne_times[dftsne_times['labels'].str.match('t')]
    
    if(label_count == 5):
        unknown_df = dftsne_times[dftsne_times['labels'].str.match('unknown')]
        unk_newlabel = unknown_df.loc[unknown_df['Time'] == unknown_df.Time.max(), 'cluster'].values[0]
        dftsne_kmeans = dftsne_kmeans.replace({'unknown': unk_newlabel})
        
    m_newlabel = m_df.loc[m_df['Time'] == m_df.Time.max(), 'cluster'].values[0]
    po_newlabel = po_df.loc[po_df['Time'] == po_df.Time.max(),'cluster'].values[0] 
    o_newlabel = o_df.loc[o_df['Time'] == o_df.Time.max(), 'cluster'].values[0]
    t_newlabel = t_df.loc[t_df['Time'] == t_df.Time.max(), 'cluster'].values[0]
    
    dftsne_kmeans = dftsne_kmeans.replace({'m': m_newlabel,
                                           'p-o': po_newlabel,
                                           'o': o_newlabel,
                                           't': t_newlabel})
    return dftsne_kmeans


In [42]:
df = pd.read_pickle(r"C:\Python\Projects\crystal-phase-prediction"
                    r"\pkl_files\structure_df_hfo2_La_del_ang.pkl")

df_labels = pd.read_pickle(r"C:\Python\Projects\crystal-phase-prediction"
                    r"\pkl_files\structure_labels_La_defects.pkl")

dotants = pd.read_pickle(r'C:\Python\Projects\crystal-phase-prediction'
                         r'\pkl_files\probe_data.pkl')
dot = dotants['dopant']
dot = np.unique(dot)
dot = dot.tolist()
dot.remove('Hf')
dot.remove('La')

labels_true = df_labels['labels_0_3']
#df = df.drop(columns=['labels_0_3'])


In [79]:
df

,structure,name,xrd_0,xrd_1,xrd_2,xrd_3,xrd_4,xrd_5,xrd_6,xrd_7,...,xrd_51,xrd_52,xrd_53,xrd_54,xrd_55,xrd_56,xrd_57,xrd_58,xrd_59,xrd_60
0,"[[1.2926438 5.17937647 1.26806526] Hf, [1.293...",Hf14La2O32_sc122_No137_1.cif,2.580160e-13,3.098205e-08,7.959790e-05,4.375433e-03,5.145982e-03,1.294921e-04,6.971825e-08,8.031144e-13,...,5.525470e-05,2.100397e-03,0.002097,0.015750,0.042089,0.019523,0.001223,0.007674,0.001314,4.817205e-06
1,[[1.29372072e+00 3.46008100e-03 6.36941120e+00...,Hf14La2O32_sc122_No137_2.cif,5.796260e-15,2.264951e-09,1.549995e-05,1.857645e-03,3.899023e-03,1.433209e-04,9.226231e-08,1.040159e-12,...,2.416214e-07,1.471635e-04,0.001747,0.013847,0.048146,0.016659,0.001095,0.007947,0.001181,3.074430e-06
2,[[ 1.29558313e+00 -6.19716000e-04 6.35358822e...,Hf14La2O32_sc122_No137_3.cif,2.681142e-47,8.320183e-36,5.011634e-26,5.859481e-18,1.329757e-11,5.857587e-07,5.008395e-04,8.312119e-03,...,7.954936e-08,7.712552e-05,0.002015,0.016820,0.052373,0.015007,0.002174,0.006152,0.002088,5.219796e-04
3,"[[1.21918514 0.09170899 1.35991767] La, [1.557...",Hf14La2O32_sc122_No137_4.cif,2.775057e-12,9.082332e-08,9.795692e-05,3.481650e-03,4.078002e-03,1.579273e-04,1.839633e-04,2.135783e-03,...,1.649348e-03,1.561519e-02,0.029646,0.018306,0.032660,0.018901,0.002708,0.006597,0.006055,8.442614e-04
4,"[[-3.75572085 -0.03552728 -8.8751733 ] La, [-1...",Hf14La2O32_sc122_No137_5.cif,1.158671e-11,1.911131e-07,1.415999e-04,4.712616e-03,7.044846e-03,4.769848e-04,7.960788e-04,7.136411e-03,...,2.951981e-03,1.837168e-02,0.027471,0.017437,0.046918,0.011570,0.003645,0.003085,0.003698,6.045090e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,"[[-8.88662646 -2.59236768 -8.88653895] La, [-8...",La_HfO2_inter_6.25_212_t.cif,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.521853e-14,3.738043e-08,0.000297,0.018125,0.037172,0.013837,0.000746,0.007136,0.000547,3.133704e-07
335,"[[ 4.92458826 -7.84505755 -1.75051193] La, [ ...",La_HfO2_inter_6.25_221_m.cif,2.524509e-14,1.537292e-09,3.250459e-06,2.386393e-04,6.083424e-04,5.449755e-05,1.873689e-04,1.871056e-03,...,5.318698e-02,1.750533e-02,0.011635,0.012808,0.036400,0.015251,0.005954,0.012817,0.012110,1.022739e-02
336,"[[5.98501319 2.70166003 0.45670373] La, [1.344...",La_HfO2_inter_6.25_221_o.cif,5.145429e-127,5.784106e-108,1.667049e-90,1.231849e-74,2.333804e-60,1.133622e-47,1.411789e-36,4.507841e-27,...,2.806020e-02,6.940285e-02,0.014662,0.012714,0.026330,0.006274,0.003351,0.008924,0.004938,1.593817e-03
337,"[[-9.00353032 2.75777889 -4.16999507] La, [-9...",La_HfO2_inter_6.25_221_p-o.cif,8.116706e-15,1.788904e-09,7.911266e-06,7.020320e-04,1.250028e-03,4.466163e-05,3.201852e-08,4.605958e-13,...,2.221232e-02,3.239151e-02,0.014053,0.012845,0.019040,0.022822,0.007428,0.016604,0.009777,6.097139e-04


In [43]:
xrd = XRDPowderPattern(two_theta_range=(5, 65))
df = xrd.fit_featurize_dataframe(df, 'structure')

X = df.iloc[:, 2:]

In [44]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [45]:
perplexity = list(range(1, 80, 1))  # perplexity values to test
labels_count = 4  # select number of lables

performance = []
for perplex in perplexity:
    dftsne = tsne_algo(X, perplex, labels_true)
    dftsne_kmeans = kmeans_algo(dftsne)
    dftsne_kmeans = sort_clusterlabels(dftsne_kmeans, labels_count)
    acc = accuracy_score(dftsne_kmeans['labels'], dftsne_kmeans['cluster'])
    performance.append(acc)
    
perf_dic = dict(zip(perplexity, performance))
print('Best value of performance: '
      + str(max(perf_dic.values()))
      + ' Perplexity = '
      + str(max(perf_dic, key=perf_dic.get)))

print(perf_dic)

Best value of performance: 0.8348082595870207 Perplexity = 19
{1: 0.3333333333333333, 2: 0.6312684365781711, 3: 0.40707964601769914, 4: 0.4336283185840708, 5: 0.5634218289085545, 6: 0.5221238938053098, 7: 0.6076696165191741, 8: 0.6873156342182891, 9: 0.6283185840707964, 10: 0.6371681415929203, 11: 0.672566371681416, 12: 0.7227138643067846, 13: 0.6283185840707964, 14: 0.6283185840707964, 15: 0.6784660766961652, 16: 0.7138643067846607, 17: 0.672566371681416, 18: 0.6755162241887905, 19: 0.8348082595870207, 20: 0.7197640117994101, 21: 0.7404129793510325, 22: 0.6873156342182891, 23: 0.6607669616519174, 24: 0.7197640117994101, 25: 0.5899705014749262, 26: 0.5929203539823009, 27: 0.5752212389380531, 28: 0.7197640117994101, 29: 0.696165191740413, 30: 0.5545722713864307, 31: 0.6784660766961652, 32: 0.7109144542772862, 33: 0.5988200589970502, 34: 0.672566371681416, 35: 0.5398230088495575, 36: 0.7463126843657817, 37: 0.5634218289085545, 38: 0.6873156342182891, 39: 0.6696165191740413, 40: 0.6578171

In [46]:
fig = px.scatter_3d(dftsne, x='x1', y='x2', z='x3', color=dftsne['labels'], 
                    title='TSNE 3D Ground Truth')
fig.show()

In [47]:
fig = px.scatter_3d(dftsne, x='x1', y='x2', z='x3', color=dftsne['cluster'], 
                    title='TSNE 3D Ground Truth')
fig.show()


In [80]:
# Hyperparameter Testing
def nmf_algo(X, comp):
    from sklearn.decomposition import NMF

    model = NMF(n_components=comp, init='random', random_state=0, max_iter = 8000)
    nmf_features_W = model.fit_transform(X)
    nmf_componentes_H = model.components_
    nmf_df = pd.DataFrame(nmf_componentes_H.T)
    W_df = pd.DataFrame(nmf_features_W) # weights represent abundence of phase at a given nominal composition
    return W_df

In [89]:
from sklearn.metrics import accuracy_score
components = list(range(3, 50 , 1))
performance = []
i = 1
for component in components: 
    W_df = nmf_algo(X, component)
    W_df_kmeans = kmeans_algo(W_df)
    W_df_kmeans = sort_clusterlabels(W_df_kmeans, 4)
    acc = accuracy_score(W_df_kmeans['labels'], W_df_kmeans['cluster'])
    performance.append(acc)
    print("Round: " + str(i))
    i = i + 1
       
perf_dic = dict(zip(components, performance))
print('Best value of performance: ' + str(max(perf_dic.values())) + ' Components = ' + str(max(perf_dic, key=perf_dic.get)))

Round: 1
Round: 2
Round: 3
Round: 4
Round: 5
Round: 6
Round: 7
Round: 8
Round: 9
Round: 10
Round: 11
Round: 12
Round: 13
Round: 14
Round: 15
Round: 16
Round: 17
Round: 18
Round: 19
Round: 20
Round: 21
Round: 22
Round: 23
Round: 24
Round: 25
Round: 26
Round: 27
Round: 28
Round: 29
Round: 30
Round: 31
Round: 32
Round: 33
Round: 34
Round: 35
Round: 36
Round: 37
Round: 38
Round: 39
Round: 40
Round: 41
Round: 42
Round: 43
Round: 44
Round: 45
Round: 46
Round: 47
Best value of performance: 0.9587020648967551 Components = 5


In [82]:
components = 5
W_df = nmf_algo(X, components)
W_df_kmeans = kmeans_algo(W_df)

names = []
for i in range(1, components +1): 
    names.append( 'x' + str(i))

signs = ['cluster', 'labels'] 
W_df_kmeans.columns = names + signs
W_df_kmeans

,x1,x2,x3,x4,x5,cluster,labels
0,0.000000,0.810441,0.081738,0.334968,0.000000,3,t
1,0.000000,0.822409,0.092329,0.213510,0.000000,3,t
2,0.000000,0.861121,0.120833,0.215632,0.000000,3,t
3,0.047541,0.720460,0.217542,0.640203,0.000000,3,t
4,0.000000,0.608734,0.349591,0.830881,0.000000,3,t
...,...,...,...,...,...,...,...
334,0.000000,0.790675,0.118885,0.000000,0.000000,3,t
335,0.978925,0.000000,0.140243,0.111413,0.051271,1,m
336,0.106494,0.153028,0.918500,0.000000,0.000000,0,o
337,0.654386,0.185501,0.174821,0.081362,0.000000,1,p-o


In [83]:
fig = px.scatter_3d(
    W_df_kmeans, x='x1', y='x2', z='x3', color=W_df_kmeans['cluster'],
    labels={'x1': 'PC 1', 'x2': 'PC 2', 'x3': 'PC 3'}, title='NMF 3D Prediction'
)
fig.show()

In [84]:
import plotly.express as px
fig = px.scatter_3d(
    W_df_kmeans, x='x1', y='x2', z='x3', color=W_df_kmeans['labels'],
    labels={'x1': 'PC 1', 'x2': 'PC 2', 'x3': 'PC 3'}, title='NMF 3D Ground Truth'
)
fig.show()

In [88]:
W_df_kmeans = sort_clusterlabels(W_df_kmeans, 4)
acc = accuracy_score(W_df_kmeans['labels'], W_df_kmeans['cluster'])
print('Acc : ' + str(acc))

AttributeError: Can only use .str accessor with string values!